In [1]:
import random
import numpy as  np
import math

#range=[0,4] i.e 5 sized grid
size=5
size-=1
def fit(particle,target):
    return 1/(1+math.sqrt(abs(target[0]-particle[0])**2+abs(target[1]-particle[1])**2))

print(fit((2,1),(4,3)))


0.2612038749637414


In [57]:
veh=[]

for _ in range(100):
    i=random.randint(0,size+1)
    j=random.randint(0,size+1)
    x=random.randint(0,size+1)
    y=random.randint(0,size+1)
    if i==size+1:
        i,j=0,0
    elif j==size+1:
        i,j=2,size
    if x==size+1:
        x,y=0,0
    elif y==size+1:
        x,y=2,size
    while (i,j)==(x,y) :
        j=random.randint(0,size)
    veh.append(((i,j),(x,y),_))
    
print(veh)

[((0, 0), (2, 0), 0), ((2, 1), (0, 0), 1), ((4, 2), (2, 3), 2), ((3, 0), (1, 3), 3), ((1, 2), (2, 4), 4), ((2, 4), (0, 2), 5), ((0, 2), (4, 4), 6), ((2, 1), (2, 0), 7), ((2, 4), (0, 2), 8), ((1, 1), (2, 3), 9), ((4, 1), (3, 0), 10), ((2, 3), (2, 2), 11), ((3, 4), (2, 0), 12), ((0, 0), (2, 0), 13), ((1, 4), (4, 4), 14), ((1, 3), (0, 2), 15), ((0, 0), (0, 2), 16), ((3, 1), (3, 4), 17), ((1, 2), (0, 0), 18), ((3, 4), (4, 2), 19), ((4, 3), (2, 3), 20), ((2, 1), (0, 0), 21), ((4, 2), (2, 4), 22), ((3, 3), (0, 3), 23), ((0, 0), (4, 4), 24), ((2, 1), (0, 0), 25), ((0, 2), (3, 0), 26), ((0, 0), (0, 1), 27), ((2, 1), (2, 4), 28), ((0, 0), (2, 0), 29), ((0, 0), (4, 1), 30), ((4, 0), (0, 0), 31), ((0, 0), (2, 4), 32), ((2, 4), (3, 0), 33), ((2, 3), (2, 4), 34), ((0, 0), (1, 0), 35), ((2, 3), (1, 0), 36), ((0, 2), (0, 0), 37), ((4, 2), (0, 0), 38), ((4, 2), (1, 3), 39), ((2, 3), (2, 1), 40), ((4, 0), (2, 0), 41), ((3, 3), (0, 0), 42), ((0, 1), (4, 0), 43), ((2, 0), (1, 4), 44), ((0, 0), (3, 1), 45

In [3]:
#for sample testing of search particles
veh=[]

for _ in range(5):
    i=random.randint(0,5)
    j=random.randint(0,5)
    x=random.randint(0,5)
    y=random.randint(0,5)
    while (i,j)==(x,y) :
        j=random.randint(0,5)
    veh.append(((i,j),(x,y),_))
    
print(veh)

[((0, 1), (2, 4), 0), ((3, 5), (2, 3), 1), ((1, 2), (1, 3), 2), ((3, 4), (3, 3), 3), ((1, 5), (1, 0), 4)]


In [64]:
count=0

def find_path(particles,target):
    global count
    path=[]
    best=[0,0,0]
    #for particle in particles:
        #print(particle)
    for particle in particles:
        cur=[(particle[0],particle[1])]
        if particle[0]+particle[2]>size or particle[0]+particle[2]<0:
            particle[2]=0
        if particle[1]+particle[3]>size or particle[1]+particle[3]<0:
            particle[3]=0
        particle[0]+=particle[2]
        particle[1]+=particle[3]
        f=fit(particle,target)
        if f>best[0]:
            best[:]=f,particle[0],particle[1]
        cur.append((particle[0],particle[1]))
        path.append(cur)
        #print(cur,'-')
    #print('path=',path)
    #print(particles)
    found=4
    cbest=[0,0,0]
    x=40
    while found==4:
        #print('iteration')
        best=cbest
        for i in range(4):
            xp,yp=target[0]-particles[i][0],target[1]-particles[i][1]
            chk=math.sqrt(xp**2+yp**2)
            if chk>1:
                n=1/chk
                xp,yp=xp*n,yp*n
            xg,yg=best[1]-particles[i][0],best[2]-particles[i][1]
            chk=math.sqrt(xg**2+yg**2)
            if chk>1:
                n=1/chk
                xg,yg=xg*n,yg*n
            xres,yres=particles[i][2]+xp+xg,particles[i][3]+yp+yg
            chk=math.sqrt(xres**2+yres**2)
            if chk>1:
                n=1/chk
                xres,yres=xres*n,yres*n
            particles[i][2],particles[i][3]=xres,yres
            if particles[i][0]+particles[i][2]>size or particles[i][0]+particles[i][2]<0:
                particles[i][2]=0
            if particles[i][1]+particles[i][3]>size or particles[i][1]+particles[i][3]<0:
                particles[i][3]=0
            particles[i][0],particles[i][1]=particle[0]+particle[2],particle[1]+particle[3]
            pbest=[particles[i][0]+xp,particles[i][1]+yp]
            f=fit(pbest,target)
            if f>cbest[0]:
                cbest[1:]=particles[i][:2]
                cbest[0]=f
            f=fit(particles[i],target)
            #print('particle ',i,'at ', particles[i][:2],', fitness=',f)
            path[i].append((particles[i][0],particles[i][1]))
            if f>=0.9:
                found=i
            #print(f)
        best=cbest
        #print(best)
        x-=1
        if x==0 :
            break
    #print("found=",found)
    if found==4:
        count+=1
        return [(0,0),(0,0)]
    return path[found]
    

particle=[]
boid_paths=[]
for v in veh:
    path_search_particles=[]
    i,j=v[0]
    #print(v)
    for _ in range(4):
        x=random.random()-0.5
        y=random.random()-0.5
        r=math.sqrt(x**2+y**2)
        n=1/r
        search_particle=[i,j,x*n,y*n]
        path_search_particles.append(search_particle)
        #print(search_particle[2],search_particle[3],'.')
    #print(path_search_particles)
    path=find_path(path_search_particles,v[1])
    #print(path)
    boid_paths.append(path)
    
print(count)

0


In [65]:
lanes=[[[100],[100],[100],[100]] for _ in range(size*size)]
print(lanes)

[[[100], [100], [100], [100]], [[100], [100], [100], [100]], [[100], [100], [100], [100]], [[100], [100], [100], [100]], [[100], [100], [100], [100]], [[100], [100], [100], [100]], [[100], [100], [100], [100]], [[100], [100], [100], [100]], [[100], [100], [100], [100]], [[100], [100], [100], [100]], [[100], [100], [100], [100]], [[100], [100], [100], [100]], [[100], [100], [100], [100]], [[100], [100], [100], [100]], [[100], [100], [100], [100]], [[100], [100], [100], [100]]]


In [65]:
signal=[]
for _ in range((size+1)*(size+1)):
    signal.append([1,0,5,5,5,5])

print(signal)

[[1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5]]


In [75]:
vehpos=[]
for v in veh:
    vehpos.append((list(v[0]),v[1],v[2]))
    
lanes=[[[-1],[-1],[-1],[-1]] for _ in range((size+1)*(size+1))]
#print(lanes)
#print(veh[0])
#print(vehpos)

for vehicle in vehpos:
    x,y=vehicle[1][0]-vehicle[0][0],vehicle[1][1]-vehicle[0][1]
    #print(vehicle)
    h,v=1,1
    if x<0:
        v=-1
    if y<0:
        h=-1
    ch=math.sqrt(abs(x))/(math.sqrt(abs(x))+math.sqrt(abs(y)))
    '''
    print(vehicle[0][0])
    print(vehicle[0][1])
    print(vehicle[2])
    print(vehicle[0][0]*5+vehicle[0][1],len(lanes))
    print(lanes[vehicle[0][0]*5+vehicle[0][1]][0])
    '''
    if random.random()<=ch:
        choice=0
        vehicle[0][0]+=v
        if v==1:
            lanes[vehicle[0][0]*size+vehicle[0][1]][2].append(vehicle[2])
        else:
            lanes[vehicle[0][0]*size+vehicle[0][1]][0].append(vehicle[2])
    else:
        choice=1
        vehicle[0][1]+=h
        if h==1:
            lanes[vehicle[0][0]*size+vehicle[0][1]][3].append(vehicle[2])
        else:
            lanes[vehicle[0][0]*size+vehicle[0][1]][1].append(vehicle[2])
    #print(vehicle)

#print(lanes)
#print(vehpos)
count=0
cost=100
#print(signal)
for _ in range(200):
    for i in range(len(lanes)):
        signal[i][0]-=1
        if signal[i][0]==0:
            signal[i][1]+=1
            signal[i][1]%=4
            signal[i][0]=signal[i][2+signal[i][1]]
        #print(i,signal[i])
        if len(lanes[i][signal[i][1]])>1:
            vh=lanes[i][signal[i][1]][1]
            #print(vh,lanes[i][signal[i][1]],end='')
            lanes[i][signal[i][1]].pop(1)
            #print(lanes[i][signal[i][1]])
            #print(vehpos[vh],'->',end='')
            x,y=vehpos[vh][1][0]-vehpos[vh][0][0],vehpos[vh][1][1]-vehpos[vh][0][1]
            #print(vehpos[vh][1],' , ',vehpos[vh][0])
            #print(vehpos[vh][1][0],'-',vehpos[vh][0][0],' , ',vehpos[vh][1][1],'-',vehpos[vh][0][1])
            #print(x,y)
            if x!=0 or y!=0:                                 #x for vertical, y for horizontal
                h,v=1,1
                if x<0:
                    v=-1
                if y<0:
                    h=-1
                ch=math.sqrt(abs(x))/(math.sqrt(abs(x))+math.sqrt(abs(y)))
                if random.random()<=ch:
                    vehpos[vh][0][0]+=v
                    if v==1:
                        lanes[vehpos[vh][0][0]*(size+1)+vehpos[vh][0][0]][2].append(vh)
                    else:
                        lanes[vehpos[vh][0][0]*(size+1)+vehpos[vh][0][0]][0].append(vh)
                else:
                    vehpos[vh][0][1]+=h
                    if h==1:
                        lanes[vehpos[vh][0][0]*(size+1)+vehpos[vh][0][0]][3].append(vh)
                    else:
                        lanes[vehpos[vh][0][0]*(size+1)+vehpos[vh][0][0]][1].append(vh)
                #print(vehpos[vh])
            else:
                count+=1
        #else:
            #print('empty')
    cost+=(100-count)
    if count==100:
        break
                        
print(count)
print(cost)

100
4581


In [76]:
#print(boid_paths)
#mod_lanes=[[1,0,1,1,1,1] for _ in range((size+1)*(size+1))]
mod_lanes=np.zeros(((size+1)*(size+1),4))
#print(size)
#print(len(mod_lanes))

for boid_path in boid_paths:
    tmp_w=np.zeros(((size+1)*(size+1),4))
    #print(tmp_w,'###')
    for z in range(len(boid_path)-1):
        x,y=boid_path[z+1][0]-boid_path[z][0],boid_path[z+1][1]-boid_path[z][1]
        for i in range(size+1):
            for j in range(size+1):
                #print(i,j,end='')
                f=fit(boid_path[z],[i,j])
                #print(f)
                if f>0.25 and f!=1:
                    v=0
                    if x>0:
                        v=2
                    tmp_w[i*size+j][v]=max(tmp_w[i*(size+1)+j][v],abs(x)*(f**2))
                    h=1
                    if x>0:
                        h=3
                    tmp_w[i*size+j][h]=max(tmp_w[i*(size+1)+j][h],abs(y)*(f**2))
    #print(tmp_w,'----------------------------------------------------')
    mod_lanes+=tmp_w

print(mod_lanes)

[[ 5.33700243  7.60645061  8.70419528  6.65774165]
 [ 5.95011239 13.03690445 11.03273268 10.81777948]
 [ 6.33527929 13.89060769  9.28083081 10.12586813]
 [ 5.41530414  9.99638759  8.0069251   7.53691831]
 [14.26371369 12.60891971 15.89898784 10.04276844]
 [11.4162178  13.32352569 12.73963873 10.91301945]
 [12.46794984 13.46431179 12.76390302 11.02803784]
 [14.23294924 12.13865727 12.91095153  9.71544063]
 [13.73627242 12.31574284 14.28607313 10.27545603]
 [12.02046766 11.27740774 11.02593531  9.98725986]
 [13.79893736 11.6205721  14.78706171 10.48985658]
 [11.609618   11.11598538 10.89366572 10.20561614]
 [11.14311741  9.73344934 10.8653758  10.0096996 ]
 [ 8.95842263  7.13085626  9.3601144   8.45125931]
 [ 7.31596099  7.67473024  8.01234064  7.19490588]
 [ 8.0765611   7.75506087  7.52366644  7.1348493 ]
 [ 8.05472466  6.6972575   7.93812129  6.08667268]
 [ 4.70727272  3.84705238  3.72508949  4.39196866]
 [ 4.95268745  4.72484858  4.30183486  4.37998423]
 [ 4.80796993  5.0131665   4.18

In [77]:
mod_lanes=mod_lanes/4

In [78]:
signal=[]
for _ in range((size+1)*(size+1)):
    signal.append([1,0,1+round(mod_lanes[_][0]),1+round(mod_lanes[_][1]),1+round(mod_lanes[_][2]),1+round(mod_lanes[_][3])])

print(signal)
print(len(signal))

[[1, 0, 2.0, 3.0, 3.0, 3.0], [1, 0, 2.0, 4.0, 4.0, 4.0], [1, 0, 3.0, 4.0, 3.0, 4.0], [1, 0, 2.0, 3.0, 3.0, 3.0], [1, 0, 5.0, 4.0, 5.0, 4.0], [1, 0, 4.0, 4.0, 4.0, 4.0], [1, 0, 4.0, 4.0, 4.0, 4.0], [1, 0, 5.0, 4.0, 4.0, 3.0], [1, 0, 4.0, 4.0, 5.0, 4.0], [1, 0, 4.0, 4.0, 4.0, 3.0], [1, 0, 4.0, 4.0, 5.0, 4.0], [1, 0, 4.0, 4.0, 4.0, 4.0], [1, 0, 4.0, 3.0, 4.0, 4.0], [1, 0, 3.0, 3.0, 3.0, 3.0], [1, 0, 3.0, 3.0, 3.0, 3.0], [1, 0, 3.0, 3.0, 3.0, 3.0], [1, 0, 3.0, 3.0, 3.0, 3.0], [1, 0, 2.0, 2.0, 2.0, 2.0], [1, 0, 2.0, 2.0, 2.0, 2.0], [1, 0, 2.0, 2.0, 2.0, 2.0], [1, 0, 2.0, 2.0, 2.0, 2.0], [1, 0, 1.0, 1.0, 1.0, 1.0], [1, 0, 1.0, 1.0, 1.0, 1.0], [1, 0, 1.0, 1.0, 1.0, 1.0], [1, 0, 1.0, 1.0, 1.0, 1.0]]
25


In [88]:
vehpos=[]
for v in veh:
    vehpos.append((list(v[0]),v[1],v[2]))
    
lanes=[[[-1],[-1],[-1],[-1]] for _ in range((size+1)*(size+1))]
#print(lanes)
#print(veh[0])
#print(vehpos)

for vehicle in vehpos:
    x,y=vehicle[1][0]-vehicle[0][0],vehicle[1][1]-vehicle[0][1]
    #print(vehicle)
    h,v=1,1
    if x<0:
        v=-1
    if y<0:
        h=-1
    ch=math.sqrt(abs(x))/(math.sqrt(abs(x))+math.sqrt(abs(y)))
    '''
    print(vehicle[0][0])
    print(vehicle[0][1])
    print(vehicle[2])
    print(vehicle[0][0]*5+vehicle[0][1],len(lanes))
    print(lanes[vehicle[0][0]*5+vehicle[0][1]][0])
    '''
    if random.random()<=ch:
        choice=0
        vehicle[0][0]+=v
        if v==1:
            lanes[vehicle[0][0]*size+vehicle[0][1]][2].append(vehicle[2])
        else:
            lanes[vehicle[0][0]*size+vehicle[0][1]][0].append(vehicle[2])
    else:
        choice=1
        vehicle[0][1]+=h
        if h==1:
            lanes[vehicle[0][0]*size+vehicle[0][1]][3].append(vehicle[2])
        else:
            lanes[vehicle[0][0]*size+vehicle[0][1]][1].append(vehicle[2])
    #print(vehicle)

for i in range(len(lanes)):
    maxlane=len(lanes[i][0])
    for j in range(1,3):
        if len(lanes[i][j])>maxlane:
            signal[i][1]=j
    signal[i][0]=min(len(lanes[i][signal[i][1]]),signal[i][signal[i][1]+2])
    
#print(lanes)
#print(vehpos)
count=0
cost=100
print(signal)
for _ in range(500):
    for i in range(len(lanes)):
        signal[i][0]-=1
        if signal[i][0]<1:
            signal[i][1]+=1
            signal[i][1]%=4
            signal[i][0]=signal[i][2+signal[i][1]]
        #print(i,signal[i])
        if len(lanes[i][signal[i][1]])>1:
            vh=lanes[i][signal[i][1]][1]
            #print(vh,lanes[i][signal[i][1]],end='')
            lanes[i][signal[i][1]].pop(1)
            #print(lanes[i][signal[i][1]])
            #print(vehpos[vh],'->',end='')
            x,y=vehpos[vh][1][0]-vehpos[vh][0][0],vehpos[vh][1][1]-vehpos[vh][0][1]
            #print(vehpos[vh][1],' , ',vehpos[vh][0])
            #print(vehpos[vh][1][0],'-',vehpos[vh][0][0],' , ',vehpos[vh][1][1],'-',vehpos[vh][0][1])
            #print(x,y)
            if x!=0 or y!=0:                                 #x for vertical, y for horizontal
                h,v=1,1
                if x<0:
                    v=-1
                if y<0:
                    h=-1
                ch=math.sqrt(abs(x))/(math.sqrt(abs(x))+math.sqrt(abs(y)))
                if random.random()<=ch:
                    vehpos[vh][0][0]+=v
                    if v==1:
                        lanes[vehpos[vh][0][0]*(size+1)+vehpos[vh][0][0]][2].append(vh)
                    else:
                        lanes[vehpos[vh][0][0]*(size+1)+vehpos[vh][0][0]][0].append(vh)
                else:
                    vehpos[vh][0][1]+=h
                    if h==1:
                        lanes[vehpos[vh][0][0]*(size+1)+vehpos[vh][0][0]][3].append(vh)
                    else:
                        lanes[vehpos[vh][0][0]*(size+1)+vehpos[vh][0][0]][1].append(vh)
                #print(vehpos[vh])
            else:
                count+=1
        #else:
            #print('empty')
    cost+=(100-count)
    if count==100:
        break
                        
print(count)
print(cost)

[[1, 0, 2.0, 3.0, 3.0, 3.0], [2, 1, 2.0, 4.0, 4.0, 4.0], [3, 1, 3.0, 4.0, 3.0, 4.0], [1, 2, 2.0, 3.0, 3.0, 3.0], [5.0, 2, 5.0, 4.0, 5.0, 4.0], [2, 3, 4.0, 4.0, 4.0, 4.0], [2, 2, 4.0, 4.0, 4.0, 4.0], [3, 2, 5.0, 4.0, 4.0, 3.0], [1, 3, 4.0, 4.0, 5.0, 4.0], [2, 2, 4.0, 4.0, 4.0, 3.0], [2, 2, 4.0, 4.0, 5.0, 4.0], [4.0, 1, 4.0, 4.0, 4.0, 4.0], [2, 3, 4.0, 3.0, 4.0, 4.0], [3, 3, 3.0, 3.0, 3.0, 3.0], [3, 3, 3.0, 3.0, 3.0, 3.0], [3.0, 1, 3.0, 3.0, 3.0, 3.0], [3.0, 2, 3.0, 3.0, 3.0, 3.0], [1, 3, 2.0, 2.0, 2.0, 2.0], [1, 2, 2.0, 2.0, 2.0, 2.0], [2.0, 1, 2.0, 2.0, 2.0, 2.0], [1, 0, 2.0, 2.0, 2.0, 2.0], [1, 3, 1.0, 1.0, 1.0, 1.0], [1, 3, 1.0, 1.0, 1.0, 1.0], [1, 3, 1.0, 1.0, 1.0, 1.0], [1, 3, 1.0, 1.0, 1.0, 1.0]]
100
4180


[[1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5], [1, 0, 5, 5, 5, 5]]
